# Component Metadata Generator

This notebook processes all Angular components in the `common/components` directory and generates structured JSON metadata for each component using an LLM.

The metadata will be used by code generation agents to understand how to use these components when creating new pages or features.

## Step 1: Import Required Libraries

In [11]:
import os
import json
from pathlib import Path
from get_secrets import run_model

print("✓ Libraries imported successfully")

✓ Libraries imported successfully


## Step 2: Configure Base Path and Component Directory

In [12]:
# Define the path to the components directory
COMPONENTS_DIR = Path(r"C:\Users\ManasSanjayPakalapat\Documents\cloudangles\motherson_demo\src\app\common\components")

print(f"Components directory: {COMPONENTS_DIR}")
print(f"Directory exists: {COMPONENTS_DIR.exists()}")

# List all component directories
if COMPONENTS_DIR.exists():
    component_dirs = [d for d in COMPONENTS_DIR.iterdir() if d.is_dir()]
    print(f"\nFound {len(component_dirs)} components:")
    for comp_dir in component_dirs:
        print(f"  - {comp_dir.name}")

Components directory: C:\Users\ManasSanjayPakalapat\Documents\cloudangles\motherson_demo\src\app\common\components
Directory exists: True

Found 3 components:
  - app-button
  - app-footer
  - app-header


## Step 3: Define Function to Read All Files in a Component

This function will read all relevant files (.ts, .html, .scss) for a given component and return their contents.

In [13]:
def read_component_files(component_dir: Path):
    """
    Read all files in a component directory and return their contents.
    
    Args:
        component_dir: Path to the component directory
        
    Returns:
        dict: Dictionary mapping file names to their contents
    """
    files_content = {}
    
    # Look for TypeScript, HTML, and SCSS files (excluding spec files for brevity)
    for file_path in component_dir.iterdir():
        if file_path.is_file():
            # Skip spec files as they're tests and not relevant for metadata
            if file_path.suffix in ['.ts', '.html', '.scss']:
                try:
                    with open(file_path, 'r', encoding='utf-8') as f:
                        files_content[file_path.name] = f.read()
                except Exception as e:
                    print(f"Error reading {file_path.name}: {e}")
    
    return files_content

# Test the function with one component
test_component = COMPONENTS_DIR / "app-button"
if test_component.exists():
    test_files = read_component_files(test_component)
    print(f"✓ Successfully read {len(test_files)} files from app-button:")
    for filename in test_files.keys():
        print(f"  - {filename}")

✓ Successfully read 4 files from app-button:
  - app-button.component.html
  - app-button.component.scss
  - app-button.component.spec.ts
  - app-button.component.ts


## Step 4: Define LLM System Prompt for Metadata Extraction

This prompt instructs the LLM to analyze component code and return structured JSON metadata.

In [14]:
SYSTEM_PROMPT = """You are an expert Angular developer analyzing component code.

Your task is to analyze the provided Angular component files and extract metadata about the component.

You MUST return ONLY a valid JSON object with this exact structure:
{
    "name": "component name",
    "description": "detailed description of what this component does and where it should be used",
    "import_path": "the exact import path that should be used to import this component in other Angular modules or components",
    "id_name": "the name of the unique identifier input property for this component that will be used in other files, or null if none exists"
}

Rules:
1. The "name" should be the component class name (e.g., "AppButtonComponent")
2. The "description" should explain:
   - What the component does
   - What inputs/outputs it has
   - When and where to use it
   - Any special features or behaviors
3. The "import_path" should be the relative path from the app root (e.g., "app/common/components/app-button/app-button.component")
4. The "id_name" is the name of the unique identifier input property for this component that will be used in other files, or null if none exists. It is what we will be basically writing in the new html or ts files to reference this component.

Return ONLY the JSON object, no additional text or explanation."""

print("✓ System prompt configured")

✓ System prompt configured


## Step 5: Create Function to Process a Single Component

This function reads all files for a component, constructs a prompt, calls the LLM, and parses the JSON response.

In [17]:
async def process_component(component_dir: Path):
    """
    Process a single component by reading its files and getting metadata from LLM.
    
    Args:
        component_dir: Path to the component directory
        
    Returns:
        dict: Component metadata or None if processing fails
    """
    component_name = component_dir.name
    print(f"\n{'='*60}")
    print(f"Processing component: {component_name}")
    print(f"{'='*60}")
    
    # Read all files in the component directory
    files_content = read_component_files(component_dir)
    
    if not files_content:
        print(f"⚠ No files found for {component_name}")
        return None
    
    print(f"✓ Read {len(files_content)} files")
    
    # Construct the user message with all file contents
    user_message = f"Analyze this Angular component: {component_name}\n\n"
    user_message += "Here are all the files in this component:\n\n"
    
    for filename, content in files_content.items():
        user_message += f"--- {filename} ---\n{content}\n\n"
    
    user_message += "\nPlease provide the component metadata in the specified JSON format."
    
    print(f"✓ Constructed prompt with {len(user_message)} characters")
    print("⏳ Calling LLM...")
    
    # Call the LLM
    try:
        response = await run_model(
            system_prompt=SYSTEM_PROMPT,
            user_message=user_message
        )
        
        print(f"✓ Received response from LLM")
        
        # Parse the JSON response
        # Remove any markdown code blocks if present
        response_text = response.strip()
        if response_text.startswith("```"):
            # Extract JSON from code block
            lines = response_text.split('\n')
            response_text = '\n'.join(lines[1:-1]) if len(lines) > 2 else response_text
        
        metadata = json.loads(response_text)
        print(f"✓ Successfully parsed metadata")
        print(f"  Component: {metadata.get('name')}")
        print(f"  Import: {metadata.get('import_path')}")
        print(f"  ID Name: {metadata.get('id_name')}")
        
        # Add the actual file contents to the metadata
        metadata['html_code'] = files_content.get(f"{component_name}.component.html", "")
        metadata['scss_code'] = files_content.get(f"{component_name}.component.scss", "")
        metadata['ts_code'] = files_content.get(f"{component_name}.component.ts", "")
        
        print(f"✓ Added file contents to metadata")
        
        return metadata
        
    except json.JSONDecodeError as e:
        print(f"❌ Error parsing JSON response: {e}")

        print(f"Response was: {response[:200]}...")
        print("✓ Function process_component() defined")

        return None

    except Exception as e:
        print(f"❌ Error processing component: {e}")
        return None

## Step 6: Process All Components

Loop through all components, process each one, and collect metadata.

In [18]:
async def process_all_components():
    """
    Process all components in the components directory.
    
    Returns:
        list: List of metadata dictionaries for all components
    """
    all_metadata = []
    
    # Get all component directories
    component_dirs = [d for d in COMPONENTS_DIR.iterdir() if d.is_dir()]
    
    print(f"\n{'#'*60}")
    print(f"STARTING COMPONENT METADATA GENERATION")
    print(f"Total components to process: {len(component_dirs)}")
    print(f"{'#'*60}\n")
    
    # Process each component
    for idx, component_dir in enumerate(component_dirs, 1):
        print(f"\n[{idx}/{len(component_dirs)}] Processing {component_dir.name}...")
        
        metadata = await process_component(component_dir)
        
        if metadata:
            all_metadata.append(metadata)
            print(f"✓ Successfully added metadata for {component_dir.name}")
        else:
            print(f"⚠ Skipped {component_dir.name} due to errors")
    
    print(f"\n{'#'*60}")
    print(f"PROCESSING COMPLETE")
    print(f"Successfully processed: {len(all_metadata)}/{len(component_dirs)} components")
    print(f"{'#'*60}\n")
    
    return all_metadata

# Run the processing
all_component_metadata = await process_all_components()


############################################################
STARTING COMPONENT METADATA GENERATION
Total components to process: 3
############################################################


[1/3] Processing app-button...

Processing component: app-button
✓ Read 4 files
✓ Constructed prompt with 2293 characters
⏳ Calling LLM...
✓ Received response from LLM
✓ Successfully parsed metadata
  Component: AppButtonComponent
  Import: app/common/components/app-button/app-button.component
  ID Name: app-button
✓ Added file contents to metadata
✓ Successfully added metadata for app-button

[2/3] Processing app-footer...

Processing component: app-footer
✓ Read 4 files
✓ Constructed prompt with 1662 characters
⏳ Calling LLM...
✓ Received response from LLM
✓ Successfully parsed metadata
  Component: AppFooterComponent
  Import: app/app-footer/app-footer.component
  ID Name: app-footer
✓ Added file contents to metadata
✓ Successfully added metadata for app-footer

[3/3] Processing app-header..

## Step 7: Display and Save Results

Show the collected metadata in a readable format and save it to a JSON file.

In [19]:
# Display the results in a readable format
print("\n" + "="*80)
print("COMPONENT METADATA COLLECTION")
print("="*80 + "\n")

for idx, metadata in enumerate(all_component_metadata, 1):
    print(f"{idx}. {metadata['name']}")
    print(f"   Import Path: {metadata['import_path']}")
    print(f"   ID Name: {metadata['id_name']}")
    print(f"   Description: {metadata['description']}")
    print(f"   HTML Code: {len(metadata.get('html_code', ''))} chars")
    print(f"   SCSS Code: {len(metadata.get('scss_code', ''))} chars")
    print(f"   TS Code: {len(metadata.get('ts_code', ''))} chars")
    print()

print("="*80)

print("="*80)
print(f"Total Components: {len(all_component_metadata)}")


COMPONENT METADATA COLLECTION

1. AppButtonComponent
   Import Path: app/common/components/app-button/app-button.component
   ID Name: app-button
   Description: A reusable button component with customizable styling and behavior. Supports three visual variants (primary, secondary, danger) with hover effects and disabled states. Accepts inputs for label text, button type (button/submit/reset), disabled state, and variant style. Emits a 'clicked' event when the button is pressed (unless disabled). Should be used throughout the application for consistent button styling and behavior.
   HTML Code: 128 chars
   SCSS Code: 642 chars
   TS Code: 576 chars

2. AppFooterComponent
   Import Path: app/app-footer/app-footer.component
   ID Name: app-footer
   Description: A footer component that displays copyright information for the ELogbook Zydus application. It shows the current year dynamically and includes copyright text. The footer has a dark blue background (#34495e) with white text, cente

In [20]:
# Save metadata to a JSON file for use by code generation agents
output_file = Path("component_metadata.json")

with open(output_file, 'w', encoding='utf-8') as f:
    json.dump(all_component_metadata, f, indent=2, ensure_ascii=False)

print(f"\n✓ Metadata saved to: {output_file.absolute()}")
print(f"  File size: {output_file.stat().st_size} bytes")
print(f"  Components included: {len(all_component_metadata)}")


✓ Metadata saved to: c:\Users\ManasSanjayPakalapat\Documents\cloudangles\motherson_demo\component_metadata.json
  File size: 10403 bytes
  Components included: 3


## Step 8: View the Final JSON Structure

This is the metadata that will be passed to the code generation agent.

In [21]:
# Display the complete JSON structure
print(json.dumps(all_component_metadata, indent=2))

# Also display as a Python list for easy copying
print("\n\n# Python-formatted metadata:")
print("component_metadata =", all_component_metadata)

[
  {
    "name": "AppButtonComponent",
    "description": "A reusable button component with customizable styling and behavior. Supports three visual variants (primary, secondary, danger) with hover effects and disabled states. Accepts inputs for label text, button type (button/submit/reset), disabled state, and variant style. Emits a 'clicked' event when the button is pressed (unless disabled). Should be used throughout the application for consistent button styling and behavior.",
    "import_path": "app/common/components/app-button/app-button.component",
    "id_name": "app-button",
    "html_code": "<button\n  [type]=\"type\"\n  [disabled]=\"disabled\"\n  [class]=\"'btn btn-' + variant\"\n  (click)=\"onClick()\">\n  {{ label }}\n</button>\n",
    "scss_code": ".btn {\n  padding: 0.5rem 1.5rem;\n  border: none;\n  border-radius: 4px;\n  font-size: 1rem;\n  cursor: pointer;\n  transition: all 0.3s;\n  font-weight: 500;\n\n  &:disabled {\n    opacity: 0.6;\n    cursor: not-allowed;\n  

---

## Usage for Code Generation Agent

The metadata generated above can now be used by a code generation agent. Here's how to use it:

```python
# Load the metadata
with open('component_metadata.json', 'r') as f:
    components = json.load(f)

# Pass to code generation agent as context
context = f"""
Available Angular components:

{json.dumps(components, indent=2)}

Use these components when generating new pages or features.
Import them using the specified import_path.
"""
```

The code generation agent will now have complete information about:
- Component names
- When and where to use each component
- Exact import paths needed